In [ ]:
inp_dir = "/home/mrassat/STAGE/images_toulouse/crop_toulouse_2/outresults_plugin_saveinput"

config = {

    "input": {
        "dsm": f"{inp_dir}/dsm.tif",
        "clr": f"{inp_dir}/clr.tif",
        "classif": f"{inp_dir}/classif.tif",
        "dtm_mesh": f"{inp_dir}/one_two/out_plugin_mesh/dtm_mesh.obj",
        "dtm_mesh_epsg": 4978
    },

    "parameters": {
        "seuil_ndvi": 0.55,
        "seuil_ndwi": -0.4
    },

    "output": {
        "out_path": f"{inp_dir}/one_two/out_plugin_mesh/",
    }
    
}

In [ ]:
import rasterio as rio
from rasterio.plot import reshape_as_raster
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:

def compute_ndvi(nir, red):
    return ( nir - red ) / ( nir + red + 0.0001 )

def compute_ndwi(nir, green):
    return ( nir - green ) / ( nir + green + 0.0001 )

def seuil_image(data, seuil, over=True):
    data_int = np.zeros( data.shape + (1,), dtype=np.uint8 )
    if over:
        data_int[data > seuil] = 255
    else:
        data_int[data < seuil] = 255
    return data_int

def to_rgba(data):

    return np.concatenate((
        data,
        data,
        data,
        data,
    ), axis = -1).astype(int) * 255


In [ ]:

with rio.open(config["input"]["clr"]) as inp:
    data = inp.read()

print(data.shape)


In [ ]:

ndvi = seuil_image( 
    compute_ndvi( data[3], data[0] ), config["parameters"]["seuil_ndvi"], True
)
ndvi = scipy.ndimage.binary_closing( ndvi[:,:,0], iterations=8 )
ndvi = scipy.ndimage.binary_opening( ndvi, iterations=5 )
ndvi = to_rgba( ndvi[:,:,np.newaxis] )

ndwi = seuil_image(
    compute_ndwi( data[3], data[1] ), config["parameters"]["seuil_ndwi"], False
)
ndwi = scipy.ndimage.binary_closing( ndwi[:,:,0], iterations=8 )
ndwi = scipy.ndimage.binary_opening( ndwi, iterations=5 )
ndwi = to_rgba( ndwi[:,:,np.newaxis] )

In [ ]:

with rio.open(
    "./vegetation_classif.png",
    "w",
    driver="PNG",
    height=ndvi.shape[0],
    width=ndvi.shape[1],
    count=4,
    dtype="uint8",
    nodata=None
) as out:
    out.write( reshape_as_raster( ndvi ) )
    
with rio.open(
    "./water_classif.png",
    "w",
    driver="PNG",
    height=ndwi.shape[0],
    width=ndwi.shape[1],
    count=4,
    dtype="uint8",
    nodata=None
) as out:
    out.write( reshape_as_raster( ndwi ) )